# Setting up the Data

In [7]:
import pandas as pd
import numpy as np
import scipy
import csv
import nltk

text = pd.read_csv("all_opinions.csv",encoding='ISO-8859-1')

issues = pd.read_csv("database.csv",encoding='ISO-8859-1')

privacyCaseSet = set([])
#for each row in the issues column of issues csv
issuesFile = open("database.csv")
issuesReader = csv.reader(issuesFile, delimiter=',')
for row in issuesReader:
    if row[39]:
    #if the integers value matches 30040, 50010, 50020, 50030, or 50040 (floats)
        privacyCaseSet.add(row[0])
        #add case_id value of same row to set
#print(privacyCaseSet)
#print(len(privacyCaseSet))

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
import random
from sklearn.decomposition import LatentDirichletAllocation 

csv.field_size_limit(100000000)
opinionsFile = open("all_opinions.csv")
opinionsReader = csv.reader(opinionsFile, delimiter=',')
dictionary = {}
count = 0

for r in opinionsReader:   
    
    if r[9] in privacyCaseSet:
        #count += 1 
        if r[9]+'-01' in dictionary.keys():
            if r[9]+'-02' in dictionary.keys():
                if r[9]+'-03' in dictionary.keys():
                    if r[9]+'-04' in dictionary.keys():
                        dictionary[r[9]+'-05'] = r[-1]
                    else: dictionary[r[9]+'-04'] = r[-1]
                else: dictionary[r[9]+'-03'] = r[-1]
            else: dictionary[r[9]+'-02'] = r[-1]
        else: dictionary[r[9]+'-01'] = r[-1]     
            

#print(len(dictionary))
#print(count)

with open('dict2.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
   # writer.writerow(['id','text'])
    for key, value in dictionary.items():
        writer.writerow([key, value])
        
test = pd.read_csv('dict2.csv',error_bad_lines=False,encoding='ISO-8859-1',names=['id','text'])

# Splitting data by years

In [9]:
fourSix = {}
sevenNine = {}
nineTwenty = {}

for key, value in dictionary.items():
    if key.startswith(str(194)):
        fourSix.update({key : value})
    if key.startswith(str(195)):
        fourSix.update({key : value})
    if key.startswith(str(196)):
        fourSix.update({key : value})
    if key.startswith(str(197)):
        sevenNine.update({key : value})
    if key.startswith(str(198)):
        sevenNine.update({key : value})
    if key.startswith(str(199)):
        nineTwenty.update({key : value})
    if key.startswith(str(200)):
        nineTwenty.update({key : value})
    if key.startswith(str(201)):
        nineTwenty.update({key : value})
        
testList1 = []

for entry1 in fourSix.values():
    testTokens1 = nltk.word_tokenize(entry1)
    if "privacy" in testTokens1:
        testList1.append(entry1)

#print(len(testList1))


testList2 = []

for entry2 in sevenNine.values():
    testTokens2 = nltk.word_tokenize(entry2)
    if "privacy" in testTokens2:
        testList2.append(entry2)

#print(len(testList2))


testList3 = []

for entry3 in nineTwenty.values():
    testTokens3 = nltk.word_tokenize(entry3)
    if "privacy" in testTokens3:
        testList3.append(entry3)

#print(len(testList3))

# Running the LDA

In [10]:
# num_topics = 15
#lda = LatentDirichletAllocation(n_components=num_topics, max_iter=500, learning_method='online', learning_offset=50.,random_state=1).fit(tf_data_samples)

#lda.score(tf_data_samples)

from IPython.display import display
def print_topics(model, vectorizer, top_n=10):    #### This is the code to print the topics. top_n can be changed. 
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

#print_topics(lda,tf_vectorizer)

num_features = 100
tf_vectorizer = CountVectorizer(max_df=.25, min_df=.05, max_features=num_features, stop_words='english')

yearDfs2 = [testList1, testList2, testList3]
ldaList = []

for year in yearDfs2:
    #n_samples = 200
    #data_samples = random.sample(year.text.tolist(), n_samples)
    data_samples = year
    tf_data_samples = tf_vectorizer.fit_transform(data_samples) 
    tf_feature_names = tf_vectorizer.get_feature_names()
    
    num_topics = 8
    lda = LatentDirichletAllocation(n_components=num_topics, max_iter=500, learning_method='online', learning_offset=50.,random_state=1).fit(tf_data_samples)
    lda.score(tf_data_samples)
    
    ldaList.append(lda)
    
#   print(lda.score(tf_data_samples))
#   print_topics(lda,tf_vectorizer)
    
#    print()
#    print()

# Word Intrustion

In [ ]:
#explore fragment the data, optimize variables see what we get
#test our best results with word intrusion
#see which topics are different between time periods

for ldaItem in ldaList:
    success = 0
    for idx1, topic1 in enumerate(ldaItem.components_):

            print("Topic %d:" % (idx1))
            array = [(tf_vectorizer.get_feature_names()[j])
                            for j in topic1.argsort()[:-10 - 1:-1]]

            cArray = [(tf_vectorizer.get_feature_names()[j])
                             for j in topic1.argsort()[:-50 - 1:-1]]

            set1 = []
            for c in array:
                set1.append(c)

            array = random.sample(array, 6)

            intruder = "?"

            for idx2, topic2 in enumerate(ldaItem.components_):
                if idx2 == idx1:
                    continue
                else:
                    bArray = [(tf_vectorizer.get_feature_names()[k])
                            for k in topic2.argsort()[:-10 - 1:-1]]
                    bArray = random.sample(bArray, 1)
                    if bArray in set1:
                        continue
                    else:
                        array.append(bArray[0])
                        intruder = bArray[0]
                        break

            random.shuffle(array)
            for j in range(7):
                print(str(j)+"    "+str(array[j]))
            print()
            g = input()
            if array[int(g)] == intruder:
                success += 1;
                print("success!")
            else:
                print("fail :(")

    print(success)

Topic 0:
0    injunction
1    false
2    religious
3    family
4    ___
5    testing
6    liability

